In [1]:
import pandas as pd
import numpy as np
import sys
from tqdm.auto import tqdm
import pebble
import pickle
from functools import partial
import warnings

from concurrent import futures
from function_residuals import calculate_residuals

sys.path.append("../Code")

# Import model functions
from get_current_model import get_model

from scipy.optimize import minimize

In [2]:
# Set the maximum number of parallel threads and the timeout
n_workers = 4 # Maximum number of parallel threads
timeout = 300 # Timeout for each thread in seconds

# Set the prefix to be used for logging and results files
# file_prefix = f"residuals_{datetime.now().strftime('%Y%m%d%H%M')}"
file_prefix = f"residuals_test"


In [3]:
# Load the model to get default parameter values
m = get_model(get_y0=False, verbose=False, check_consistency=False)

In [4]:
def get_fitting_startvalue(fitting_parameter, m):
    if fitting_parameter == "fluo_influence":
        return pd.Series({f"__fluo_influence__{k}":v for k,v in m.parameters[fitting_parameter].items()})
    else:
        return pd.Series({fitting_parameter: m.parameters[fitting_parameter]})
    
def get_fitting_startvalues(fitting_parameters,m):
    start_values = [get_fitting_startvalue(x,m) for x in fitting_parameters]
    return pd.concat(start_values)

def get_fitting_parameter_dict(values, names):
    # Put values into a pandas series for easier handling
    res = pd.Series(values, index=names)

    # Reconstitute fluo_influence
    fluo_influence_bool = res.index.str.startswith("__fluo_influence__")
    if fluo_influence_bool.any():
        # Extract the intermediate values
        _res = res[fluo_influence_bool]
        _res.index = _res.index.str.removeprefix("__fluo_influence__")
        
        res = res[np.invert(fluo_influence_bool)]
        res["fluo_influence"] = _res.to_dict()

    return res.to_dict()

In [5]:
fitting_parameters = [
    "fluo_influence",
    "lcf",
    "kUnquench",
    "KMUnquench",
    "kQuench",
    "kOCPactivation",
    "kOCPdeactivation",
    "OCPmax",
]

start_values = get_fitting_startvalues(fitting_parameters,m)

p, p_names = start_values.values, start_values.index

In [6]:
# Function to calculate residuals with arguments tailored to the minimize function
def calculate_residuals_minimize(p, p_names, scale_factors=None):
    # Undo the scaling
    if scale_factors is not None:
        p = p * scale_factors

    _p = get_fitting_parameter_dict(p, p_names)

    return calculate_residuals(
        _p,
        n_workers=5,
        timeout=60, # s
        logger_filename="../out/minimise",
        save_intermediates=False
        )

def fit_model_parameters(start_values, scale_to_value=None):
    
    # If the parameters should be scaled, replace them with the scaling value
    if scale_to_value is None:
        p = start_values.values
        scale_factors = None
    else:
        p = np.full(start_values.shape[0], scale_to_value)
        scale_factors = start_values.values / scale_to_value

    fit = minimize(
        fun = calculate_residuals_minimize,
        x0 = p,
        args = (start_values.index, scale_factors),
    )

    # Rescale the results
    if scale_to_value is not None:
        fit.x = fit.x * p/scale_to_value
    return fit

In [7]:
test = fit_model_parameters(start_values, 0.1)

KeyboardInterrupt: 

In [ ]:
# test = minimize(
#         fun = minimise_calculate_residuals,
#         x0 = p,
#         args = (p_names),
#     )

In [ ]:
test

In [ ]:
import logging
from pathlib import Path

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
def setup_logger(name, log_file, level=logging.INFO):
    """To setup as many loggers as you want"""

    handler = logging.FileHandler(log_file)        
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)

    return logger

In [ ]:
logger_filename = "test"

def test_logger():
    ErrorLogger = setup_logger("ErrorLogger", Path(f"{logger_filename}_err.log"), level=logging.ERROR)
    ErrorLogger.error("test")

In [ ]:
test_logger()

In [ ]:
logger_filename = "test"
ErrorLogger = setup_logger("test", Path(f"{logger_filename}_err.log"), level=logging.ERROR)
ErrorLogger.error("test")

In [ ]:
logger = logging.getLogger("test2")

In [ ]:
len(logger.handlers)

In [ ]:
logging.getHandlerByName